# Select Data

In [18]:
from spflows.configs_classes.forecasting_configs import ForecastingModelConfig
from spflows.data.datamodules import ForecastingDataModule

config = ForecastingModelConfig(prefetch_factor=None,
                                batch_size=19)
datamodule = ForecastingDataModule(config)

# Datasets

In [19]:
training_data,test_data,validation_data = datamodule.get_data()

In [20]:
training_data[0]["target"].shape

(370, 5353)

In [21]:
training_data[0]["target"].shape[0]

370

# Dataloaders

In [22]:
data_batch = next(datamodule.train_dataloader().__iter__())

In [ ]:
data_batch.keys()

dict_keys(['target_dimension_indicator', 'past_time_feat', 'past_target_cdf', 'past_observed_values', 'past_is_pad', 'future_time_feat', 'future_target_cdf', 'future_observed_values'])

In [25]:
data_batch["target_dimension_indicator"].shape

torch.Size([19, 370])